In [1]:
import asyncio
import websockets
import json
from pathlib import Path
import logging
import numpy as np
from pydub import AudioSegment
import io

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
def prepare_audio_chunks(input_file: str, chunk_size: int = 4096):
    """Read an audio file and prepare it for streaming."""
    # Load the audio file
    audio = AudioSegment.from_mp3(input_file)
    
    # Convert to wav format with required parameters
    audio = audio.set_frame_rate(16000).set_channels(1)
    
    # Convert to raw PCM data
    raw_data = np.array(audio.get_array_of_samples())
    
    # Normalize and convert to 16-bit integers
    normalized = np.int16(raw_data / np.max(np.abs(raw_data)) * 32767)
    
    # Split into chunks
    chunks = [normalized[i:i + chunk_size] for i in range(0, len(normalized), chunk_size)]
    
    # Convert chunks to bytes
    byte_chunks = [chunk.tobytes() for chunk in chunks]
    
    return byte_chunks

In [21]:
async def test_audio_pipeline(input_file: str, output_file: str):
    """Test the complete audio pipeline."""
    uri = "ws://localhost:8000/ws/transcribe"
    chunks = prepare_audio_chunks(input_file)
    received_audio_chunks = []
    
    async with websockets.connect(uri) as websocket:
        logger.info("Connected to WebSocket")
        
        
        
        # Send audio chunks
        for chunk in chunks:
            
            await websocket.send(chunk)
            await asyncio.sleep(0.1)
            logger.info(f"Sent chunk of size {len(chunk)} bytes")
        
       
        # Send stop command
        await websocket.send("STOP_PROCESS")
        logger.info("Sent STOP_PROCESS command")
        
        # Receive response
        await asyncio.sleep(5)
        while True:
            try:
                response = await websocket.recv()
                if isinstance(response, bytes):
                    logger.info(f"Received audio chunk of size {len(response)} bytes")
                    received_audio_chunks.append(response)
                else:
                    logger.info(f"Received message: {response}")
                    if response == "RESPONSE_COMPLETE":
                        break
            except websockets.exceptions.ConnectionClosed:
                logger.info("Connection closed")
                break
    
    # Combine received chunks and save
    if received_audio_chunks:
        combined_data = b''.join(received_audio_chunks)
        with open(output_file, 'wb') as f:
            f.write(combined_data)
        logger.info(f"Saved response audio to {output_file}")
    else:
        logger.warning("No audio chunks received")

In [22]:
input_file = "landmass.mp3"  # Replace with your test audio file
output_file = "test_output.mp3"

await test_audio_pipeline(input_file, output_file)

INFO:__main__:Connected to WebSocket
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 bytes
INFO:__main__:Sent chunk of size 8192 b